## Preparing OpenNMT and other dependencies

In [1]:
%cd ./drive/My Drive

/content/drive/My Drive


In [0]:
!pip install OpenNMT-py

In [0]:
!git clone https://github.com/cdli-gh/Unsupervised-NMT-for-Sumerian-English.git

Cloning into 'Unsupervised-NMT-for-Sumerian-English'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 191 (delta 0), reused 3 (delta 0), pack-reused 187
Receiving objects: 100% (191/191), 65.76 MiB | 18.89 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Checking out files: 100% (52/52), done.


In [0]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git

In [5]:
%cd OpenNMT-py/

/content/drive/My Drive/OpenNMT-py


In [0]:
!mkdir ./data/sum-en

In [0]:
%cp ../Unsupervised-NMT-for-Sumerian-English/dataset/dataToUse/* ./data/sum-en/

## Preparing Data

In [0]:
!onmt_preprocess -train_src data/sum-en/sum_train.txt -train_tgt data/sum-en/eng_train.txt -valid_src data/sum-en/sum_dev.txt -valid_tgt data/sum-en/eng_dev.txt -save_data data/sum-en/demo

Traceback (most recent call last):
  File "/usr/local/bin/onmt_preprocess", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/onmt/bin/preprocess.py", line 298, in main
    preprocess(opt)
  File "/usr/local/lib/python3.6/dist-packages/onmt/bin/preprocess.py", line 246, in preprocess
    ArgumentParser.validate_preprocess_args(opt)
  File "/usr/local/lib/python3.6/dist-packages/onmt/utils/parse.py", line 144, in validate_preprocess_args
    assert os.path.isfile(file), "Please check path of %s" % file
AssertionError: Please check path of data/sum-en/sum_train.txt


## Training

In [0]:
!python  train.py -data ./data/sum-en/demo -save_model ./data/sum-en/ \
        -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8  \
        -encoder_type transformer -decoder_type transformer -position_encoding \
        -train_steps 100000  -max_generator_batches 2 -dropout 0.1 \
        -batch_size 4096 -batch_type tokens -normalization tokens  -accum_count 2 \
        -train_from ./data/sum-en/_step_5000.pt \
        -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 \
        -max_grad_norm 0 -param_init 0  -param_init_glorot \
        -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 \
        -world_size 1 -gpu_ranks 0
        # -early_stopping 5 \

[2020-06-08 17:01:03,939 INFO] Loading checkpoint from ./data/sum-en/_step_5000.pt
[2020-06-08 17:01:05,596 INFO] Loading vocab from checkpoint at ./data/sum-en/_step_5000.pt.
[2020-06-08 17:01:05,628 INFO]  * src vocab size = 29242
[2020-06-08 17:01:05,628 INFO]  * tgt vocab size = 15062
[2020-06-08 17:01:05,628 INFO] Building model...
[2020-06-08 17:01:15,102 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(29242, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512,

## Translating test data and saving

In [0]:
!onmt_translate -model ./data/sum-en/_step_21000.pt -src ./data/sum-en/sum_test.txt -output ./data/sum-en/pred_engg1.txt -replace_unk -verbose

In [0]:
with open('./data/sum-en/sum_test.txt', 'r') as f:
  inp = f.readlines()

In [0]:
with open('./data/sum-en/eng_test.txt', 'r') as f:
  actual = f.readlines()

In [0]:
with open('./data/sum-en/pred_engg1.txt', 'r') as f:
  preds = f.readlines()

In [11]:
len(actual), len(preds), len(inp)

(12209, 2640, 12209)

In [0]:
import pandas as pd

In [0]:
to_df = {'input':inp[:len(preds)], 'actual':actual[:len(preds)], 'predicted':preds}

pd.DataFrame(to_df).to_csv('../Unsupervised-NMT-for-Sumerian-English/Results/Tranformer1Preds2.csv')

## Evaluating Translations

### BLEU

In [0]:
df = pd.read_csv('../Unsupervised-NMT-for-Sumerian-English/Results/Tranformer1Preds2.csv')

In [0]:
actual = list(df['actual'])
preds = list(df['predicted'])

In [0]:
import nltk

#there may be several references

blues = []
for i in range(len(actual)):
  if(len(preds[i].split(' ')) > 4):
    blues.append(nltk.translate.bleu_score.sentence_bleu([actual[i]], preds[i]))
  else:
    blues.append(nltk.translate.bleu_score.sentence_bleu([actual[i]], preds[i], weights=(0.5, 0.5)))

In [35]:
print(len(blues))

2640


In [36]:
import numpy as np
np.mean(blues)

0.5139389596944947

In [0]:
df1 = pd.read_csv('../../../Transform_model_results.csv')

In [39]:
df1.head()

,Unnamed: 0,Sumerian - Input,English - Exact,English - Predicted
0,NaN,NUMB sila ce sa ce inuha,"NUMB sila3 barley … straw ,",NUMB ban NUMB sila barley
1,insd,NUMB sila gugalgal mua,"NUMB sila3 , chickpeas , grown ,",NUMB sila3 NUMB shekel silver
2,NaN,NUMB sila guturtur mua,"NUMB sila3 lentils , grown ,",NUMB sila3 NUMB shekel silver
3,NaN,NUMB sila gig mua,"NUMB sila3 wheat , grown ,",NUMB sila3 wheat ;
4,NaN,NUMB sila imgaga mua,"NUMB sila3 … grown ,","NUMB , NUMB ban NUMB sila spelt"


In [0]:
actual = list(df1['English - Exact'])
preds = list(df1['English - Predicted'])

blues = []
for i in range(len(actual)):
  try:
    if(len(preds[i].split(' ')) > 4):
      blues.append(nltk.translate.bleu_score.sentence_bleu([actual[i]], preds[i]))
    else:
      blues.append(nltk.translate.bleu_score.sentence_bleu([actual[i]], preds[i], weights=(0.5, 0.5)))

  except:
    continue

In [41]:
np.mean(blues)

0.4321301180725545

### ROUGE

In [30]:
!pip install py-rouge

     |████████████████████████████████| 61kB 1.8MB/s 


In [0]:
import rouge

In [0]:
import nltk

In [44]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
actual = list(df['actual'])
preds = list(df['predicted'])

In [0]:
evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                        max_n=4,
                        limit_length=True,
                        length_limit=100,
                        length_limit_type='words',
                        apply_avg=1,
                        apply_best=0,
                        alpha=0.5, # Default F1_score
                        weight_factor=1.2,
                        stemming=False)

scores = evaluator.get_scores(preds, actual[:len(preds)])

In [73]:
scores

{'rouge-1': {'f': 0.517633252183997,
  'p': 0.5512636333894455,
  'r': 0.5125632535985445},
 'rouge-2': {'f': 0.3340806524031567,
  'p': 0.34844586233672065,
  'r': 0.33174298557307924},
 'rouge-3': {'f': 0.20964063194741886,
  'p': 0.21732137697294782,
  'r': 0.20851117829415214},
 'rouge-4': {'f': 0.13888663662192888,
  'p': 0.14255935930651842,
  'r': 0.13820415405946124},
 'rouge-l': {'f': 0.5343244191034694,
  'p': 0.5631495012573814,
  'r': 0.528058548452696},
 'rouge-w': {'f': 0.43930921832040526,
  'p': 0.5345933095251012,
  'r': 0.3961406025727287}}

In [0]:
actual = list(df['actual'])
preds = list(df['predicted'])

In [0]:
scores = evaluator.get_scores(preds[:900], actual[:900])

In [76]:
scores

{'rouge-1': {'f': 0.5086076810402741,
  'p': 0.5428226609831529,
  'r': 0.5054277544409181},
 'rouge-2': {'f': 0.3294871226408093,
  'p': 0.34511545967983004,
  'r': 0.32678033929198297},
 'rouge-3': {'f': 0.21111194090711713,
  'p': 0.22071645302772755,
  'r': 0.20836963014530094},
 'rouge-4': {'f': 0.14319100866740184,
  'p': 0.14713490059323395,
  'r': 0.1417453022124075},
 'rouge-l': {'f': 0.5249557713345762,
  'p': 0.5539110097476229,
  'r': 0.5204324577832755},
 'rouge-w': {'f': 0.4307835257929621,
  'p': 0.5260221913845455,
  'r': 0.3903121447644038}}